In [ ]:
%pip install requests

In [1]:
import base64
import requests
import datetime

In [2]:
client_id = "a3257d3e8f294fac939a9c3dd128cb73"
client_secret = "45b4f34f2ba447799b81e2a4c98395cb"

In [15]:
class SpotifyAPI(object):
    # default variables
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    method = "POST"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        
        # if client_id / client_secret are not set raise exception
        if client_id == None or client_secret == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode()) # encode to byte then convert to b64
        return client_creds_b64.decode() # decode the b64 from bytes
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        }
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        
        # sends a request to Spotify API
        r = requests.post(token_url, data=token_data, headers=token_headers)
        # checks if valid request
        if r.status_code not in range(200, 299):
            return False # if not return False
        data = r.json() # our token received from Spotify
        access_token = data['access_token'] # token needed to request data from Spotify
        expires_in = data['expires_in'] # seconds until the token expires
        now = datetime.datetime.now() # datetime right now to create datetime of when token expires
        expires = now + datetime.timedelta(seconds=expires_in) # gives expiration a datetime stamp
        self.access_token = access_token
        self.access_token_expires = expires # sets access_token_expires = expires
        self.access_token_did_expire = expires < now # boolean, checks if access token expired
        return True
        

In [16]:
spotify = SpotifyAPI(client_id, client_secret)

In [17]:
spotify.perform_auth()

True

In [18]:
spotify.access_token

'BQDj46F9aSbgE9BtmXYQSrfkEprfRFkmVnWjl-nnOHGS1l5XJ1494HnK2WAOTT5K3XeV6rmxNDTzA-sjnls'